In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/archive.zip" /content/

In [3]:
!unzip -q "/content/drive/MyDrive/archive.zip" -d /content/

## Step 1: Data Preparation

In [4]:
# Step 1: Data Preparation
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split

# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (224, 224))
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
    return np.array(images)

# Load raw and reference images
raw_images = load_images_from_folder('/content/Train/Raw')
reference_images = load_images_from_folder('/content/Train/Reference')

# Split the dataset into training and testing sets
X_train, X_test = train_test_split(raw_images, test_size=0.2, random_state=42)


## Step 2: Define the VAE Architecture

In [5]:
# Step 2: Define the VAE Architecture
import tensorflow as tf
from tensorflow.keras import layers, models

# Encoder
latent_dim = 64  # Latent space dimension

encoder_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Flatten the output
x = layers.Flatten()(x)
# Mean and log variance
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Reparameterization Trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Latent space
z = layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(28 * 28 * 128, activation='relu')(decoder_input)
x = layers.Reshape((28, 28, 128))(x)
x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoder_output = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# Build VAE model
encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_input, decoder_output, name='decoder')
vae_output = decoder(encoder(encoder_input)[2])

# VAE Model
vae = models.Model(encoder_input, vae_output, name='vae')


## Step 3: Define Loss Function

In [6]:
# Step 3: Define Loss Function
class VAEModel(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAEModel, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)  # Add KL loss to the model's losses
        return reconstructed

vae_model = VAEModel(encoder, decoder)

# Compile the model
vae_model.compile(optimizer='adam', loss='mean_squared_error')


## Step 4: Training the VAE

In [7]:
# Step 4: Training the VAE
history = vae_model.fit(X_train, X_train,
                         epochs=200,
                         batch_size=16,
                         validation_data=(X_test, X_test))


Epoch 1/200
35/35 [==============================] - 22s 566ms/step - loss: 0.0617 - val_loss: 0.0464
Epoch 2/200
35/35 [==============================] - 19s 533ms/step - loss: 0.0451 - val_loss: 0.0472
Epoch 3/200
35/35 [==============================] - 19s 533ms/step - loss: 0.0449 - val_loss: 0.0444
Epoch 4/200
35/35 [==============================] - 18s 528ms/step - loss: 0.0429 - val_loss: 0.0438
Epoch 5/200
35/35 [==============================] - 18s 522ms/step - loss: 0.0432 - val_loss: 0.0450
Epoch 6/200
35/35 [==============================] - 18s 526ms/step - loss: 0.0431 - val_loss: 0.0438
Epoch 7/200
35/35 [==============================] - 18s 528ms/step - loss: 0.0424 - val_loss: 0.0430
Epoch 8/200
35/35 [==============================] - 19s 542ms/step - loss: 0.0425 - val_loss: 0.0444
Epoch 9/200
35/35 [==============================] - 18s 511ms/step - loss: 0.0426 - val_loss: 0.0434
Epoch 10/200
35/35 [==============================] - 19s 535ms/step - loss: 0.042

## Step 5: Evaluation


In [11]:
!pip install tensorflow
import os
import numpy as np
from tensorflow.image import psnr as tf_psnr, ssim as tf_ssim
from tensorflow.image import resize
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf



IMG_WIDTH = 128
IMG_HEIGHT = 128
CHANNELS = 3

RAW_TEST_PATH = '/content/Test/Raw'
REFERENCE_TEST_PATH = '/content/Test/Reference'

def load_images_from_folder(folder, img_width=224, img_height=224):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
        images.append(img_array)
    return np.array(images)

# Now, reload the images with the updated function
raw_test_images = load_images_from_folder(RAW_TEST_PATH)
reference_test_images = load_images_from_folder(REFERENCE_TEST_PATH)

predicted_images = vae_model.predict(raw_test_images)

6/6 [==============================] - 2s 371ms/step


In [12]:
predicted_images[0].shape

(224, 224, 3)

In [13]:
reference_test_images[0].shape

(224, 224, 3)

In [16]:
# Define MSE, PSNR, and SSIM functions
def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def psnr(y_true, y_pred):
    max_pixel = 1.0
    return 20 * np.log10(max_pixel / np.sqrt(mse(y_true, y_pred)))

def ssim(y_true, y_pred):
    return tf.reduce_mean(tf_ssim(y_true, y_pred, max_val=1.0))

mse_values = []
psnr_values = []
ssim_values = []

for i in range(len(predicted_images)):
    mse_val = mse(reference_test_images[i], predicted_images[i])
    psnr_val = psnr(reference_test_images[i], predicted_images[i])
    ssim_val = ssim(reference_test_images[i], predicted_images[i])

    mse_values.append(mse_val)
    psnr_values.append(psnr_val)
    ssim_values.append(ssim_val)

# Calculate the mean of the evaluation metrics over the test set
mean_mse = np.mean(mse_values)
mean_psnr = np.mean(psnr_values)
mean_ssim = np.mean(ssim_values)

print(f"Mean MSE: {mean_mse}")
print(f"Mean PSNR: {mean_psnr}")
print(f"Mean SSIM: {mean_ssim}")

Mean MSE: 0.08615907281637192
Mean PSNR: 10.881352121899013
Mean SSIM: 0.23851041495800018
